# Planet: Understanding the Amazon from Space

In [1]:
from fastai.vision import *
from tqdm import tqdm

In [2]:
path = Path('../data')
path

PosixPath('../data')

In [3]:
test = ImageList.from_folder(path/'test-jpg')
len(test)

61191

In [4]:
model_list = ['resnet50.pkl','resnet101.pkl', 'resnet152.pkl', 'densenet121.pkl', 'densenet169.pkl']

In [ ]:
total = np.zeros((len(test), 17))
for model in model_list:
    learn = load_learner(path, model, test=test)
    preds, targets = learn.TTA(scale=1.05, ds_type=DatasetType.Test)
    total = np.add(total, np.array(preds))
    
preds_avg = np.divide(total, len(model_list))

/home/ncondo/anaconda3/envs/planet-amazon/lib/python3.6/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'torchvision.models.densenet._DenseBlock' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/ncondo/anaconda3/envs/planet-amazon/lib/python3.6/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'torchvision.models.densenet._DenseLayer' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [ ]:
varying_thresh = False
if varying_thresh:
    # thresholds found after running get_thresholds.ipynb
    thresholds = [0.14,0.24,0.14,0.17,0.11,0.49,0.1,0.33,0.12,0.25,0.11,0.1,0.49,0.15,0.18,0.15,0.23]
    labelled_preds = [' '.join([learn.data.classes[i] for i,p in enumerate(pred) if p > thresholds[i]]) for pred in preds_avg]
else:
    thresh = 0.17
    labelled_preds = [' '.join([learn.data.classes[i] for i,p in enumerate(pred) if p > thresh]) for pred in preds_avg]

In [ ]:
labelled_preds[:5]

In [ ]:
fnames = [f.name[:-4] for f in learn.data.test_ds.items]

In [ ]:
sub_df = pd.DataFrame({'image_name':fnames, 'tags':labelled_preds}, columns=['image_name', 'tags'])

In [ ]:
submission_fname = 'final-ensemble.csv'

In [ ]:
sub_df.to_csv(path/'submissions'/submission_fname, index=False)

In [ ]:
! kaggle competitions submit planet-understanding-the-amazon-from-space -f {path/'submissions'/submission_fname} -m "My submission"